In [1]:
import pandas as pd
import numpy as np

## BƯỚC 1: ĐỌC DỮ LIỆU

In [4]:
# Đọc sheet 'Demand', không lấy cột B
df_demand = pd.read_excel('INPUT_supply_planning.xlsx',sheet_name='Demand',usecols='A,C,D')
df_demand.head(2)

FileNotFoundError: [Errno 2] No such file or directory: 'INPUT_supply_planning.xlsx'

In [ ]:
# Đọc sheet 'Supply', bỏ dòng đầu tiên
df_supply = pd.read_excel('INPUT_supply_planning.xlsx',sheet_name='Supply',skiprows=0,header=1)
df_supply.head(2)

## BƯỚC 2: LUỸ KẾ DEMAND

In [ ]:
# Sắp xếp data sheet Demand theo thứ tự tăng dần của Part_No và Job start date
df_demand = df_demand.sort_values(by=['Part_No','Job start date'])

In [ ]:
# Tạo cột cumsum từ Job_qty cho từng Part_no
df_demand['cumsum_demand'] = df_demand.groupby('Part_No')['Job Qty'].cumsum()
df_demand.head(2)

## BƯỚC 3: Unpivot dữ liệu Supply
TÌM SỐ LƯỢNG STOCK ON HAND VÀ STOCK ĐANG VỀ (IN-TRANSIT)

In [ ]:
# Đổi tên cột stock on hand thành ngày 1 ngày trong hiện tại hoặc quá khứ. 
# Mục đích khi cung cấp cho demand thì sẽ lấy những dòng này đầu tiên
df_supply = df_supply.rename(columns={'Stock on hand':pd.to_datetime('2000-01-01')}) #pd.Timestamp.today()

In [ ]:
# Lấy số lượng hàng in-transit
df_supply_melted = df_supply.melt(id_vars='Part_No',var_name='supply_week',value_name='supply_qty')

# Sort theo Part_no và Supply_week tăng dần
df_supply_melted = df_supply_melted.sort_values(by=['Part_No','supply_week'])
df_supply_melted.head()

In [ ]:
# Nhóm theo mã hàng, xem số lượng luỹ tiến supply là bao nhiêu
df_supply_melted['cumsum_supply'] = df_supply_melted.groupby('Part_No')['supply_qty'].cumsum()
df_supply_melted

## BƯỚC 4: JOIN DEMAND VÀ SUPPLY

In [ ]:
# Join bảng demand và bảng supply
df_demand_supply = df_demand.merge(df_supply_melted,on='Part_No')
df_demand_supply.head()

## BƯỚC 5: CHỌN SUPPLY_WEEK ĐẦU TIÊN ĐÁP ỨNG ĐƯỢC SỐ LƯỢNG

In [ ]:
# Bỏ các trường hợp supply không đủ cho demand
df_enough = df_demand_supply[df_demand_supply['cumsum_demand'] <= df_demand_supply['cumsum_supply']]

In [ ]:
# Lấy dòng đầu tiên trong mỗi demand vì đã đủ hàng supply (đủ vì đã filter ở bước kề trên)
df_first_supply = df_enough.groupby(['Part_No','cumsum_demand']).first()

In [ ]:
# Giữ lại các cột cần thiết
df_first_supply = df_first_supply.reset_index()[['Part_No','Job Qty','Job start date','supply_week','supply_qty']]
df_first_supply

## BƯỚC 6: Lấy lại các dòng không có demand hoặc supply

In [ ]:
# Lấy lại các dòng không có demand hoặc không có supply
df_final = df_first_supply.merge(df_demand,how='right')
df_final.head()

In [ ]:
df_final['late_date'] = df_final['supply_week'] - df_final['Job start date'] 
df_final.head()

In [ ]:
df_final['supply_week'] = df_final['supply_week'].fillna('no supply')
df_final = df_final.fillna('')

## BƯỚC 7: XUẤT RA FILE EXCEL

In [17]:
df_final.to_excel('OUTPUT_supply.xlsx')